#### connect to bigquery

In [1]:
from google.cloud import bigquery

client = bigquery.Client(project="sales-pulse-etl-dashboard")
project_id = 'sales-pulse-etl-dashboard'
dataset_id = 'sales_data'
table_id = 'advertising_metrics'
table_ref = f"{project_id}.{dataset_id}.{table_id}"

In [2]:
print(f"Table reference: {table_ref}")

Table reference: sales-pulse-etl-dashboard.sales_data.advertising_metrics


In [5]:
%load_ext bigquery_magics

The bigquery_magics extension is already loaded. To reload it, use:
  %reload_ext bigquery_magics


##### sql queries

In [8]:
%%bigquery result --project {project_id}
SELECT *
FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
display(result)

,TV,Radio,Newspaper,Sales,Total_Spend,TV_Percentage,Radio_Percentage,Newspaper_Percentage,TV_Radio_Interaction,TV_Newspaper_Interaction,Log_Sales,Log_TV,ROAS
0,0.7,39.6,8.7,1.6,49.0,0.014286,0.808163,0.177551,27.72,6.09,0.955511,0.530628,0.032653
1,4.1,11.6,5.7,3.2,21.4,0.191589,0.542056,0.266355,47.56,23.37,1.435085,1.629241,0.149533
2,8.6,2.1,1.0,4.8,11.7,0.735043,0.179487,0.085470,18.06,8.60,1.757858,2.261763,0.410256
3,5.4,29.9,9.4,5.3,44.7,0.120805,0.668904,0.210291,161.46,50.76,1.840550,1.856298,0.118568
4,13.1,0.4,25.6,5.3,39.1,0.335038,0.010230,0.654731,5.24,335.36,1.840550,2.646175,0.135550
5,7.3,28.1,41.4,5.5,76.8,0.095052,0.365885,0.539062,205.13,302.22,1.871802,2.116256,0.071615
6,13.2,15.9,49.6,5.6,78.7,0.167726,0.202033,0.630241,209.88,654.72,1.887070,2.653242,0.071156
7,8.4,27.2,2.1,5.7,37.7,0.222812,0.721485,0.055703,228.48,17.64,1.902108,2.240710,0.151194
8,17.2,4.1,31.6,5.9,52.9,0.325142,0.077505,0.597353,70.52,543.52,1.931521,2.901422,0.111531
9,19.4,16.0,22.3,6.6,57.7,0.336222,0.277296,0.386482,310.40,432.62,2.028148,3.015535,0.114385


average sales

In [15]:
%%bigquery result --project {project_id}
SELECT AVG(Sales) AS avg_sales, AVG(Total_Spend) AS avg_total_spend
FROM `sales-pulse-etl-dashboard.sales_data.advertising_metrics`;

Query is running:   0%|          |

Downloading:   0%|          |

In [16]:
display(result)

,avg_sales,avg_total_spend
0,14.0225,200.8605


ROAS levels

Query is running:   0%|          |

Downloading:   0%|          |

In [18]:
display(result)

,Sales,Total_Spend,ROAS,efficiency_category
0,1.6,49.0,0.032653,Low Efficiency
1,3.2,21.4,0.149533,Low Efficiency
2,4.8,11.7,0.410256,Low Efficiency
3,5.3,44.7,0.118568,Low Efficiency
4,5.3,39.1,0.135550,Low Efficiency
5,5.5,76.8,0.071615,Low Efficiency
6,5.6,78.7,0.071156,Low Efficiency
7,5.7,37.7,0.151194,Low Efficiency
8,5.9,52.9,0.111531,Low Efficiency
9,6.6,57.7,0.114385,Low Efficiency
